In [ ]:
from google.colab import files
uploaded = files.upload()

Saving OpinHuBank_Majority.csv to OpinHuBank_Majority.csv


In [ ]:
import io
import pandas as pd
df = pd.read_csv(io.BytesIO(uploaded['OpinHuBank_Majority.csv']), sep=';')

In [ ]:
df

In [ ]:
df1 = df['Sentence']
df1

In [ ]:
def iloc_label(df1,col):
  for i in df1.index:
    val = df1[col].iloc[i]
    if val == 'SEM':
      df1.at[i,col] = 1
    elif val == 'POZ':   
      df1.at[i,col] = 2
    elif val == 'NEG':
      df1.at[i,col] = 0 
      

In [ ]:
training_data = df1.sample(frac=0.8, random_state=25)
testing_data = df1.drop(training_data.index)

In [ ]:
%%capture cap

for i in testing_data.index:
  print(testing_data[i])


In [ ]:
f = open("OpinHuBank_TestEgysoros.txt", "w") 
print(cap, file=f)
f.close()  

In [ ]:
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()

In [ ]:
features = vectorizer.fit_transform(training_data.Sentence) 
features

<6160x33340 sparse matrix of type '<class 'numpy.int64'>'
	with 142711 stored elements in Compressed Sparse Row format>

In [ ]:
from sklearn.linear_model import SGDClassifier
cls = SGDClassifier()
model = cls.fit(features, training_data.LABEL)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

In [ ]:
test_features = vectorizer.transform(testing_data.Sentence) 
prediction = model.predict(test_features)

Szózsák acc:

In [ ]:
accuracy_score(y_true=testing_data.LABEL, y_pred=prediction)

0.7798701298701298

Opinhubank adatbázison futtatom a Bertet

In [ ]:
pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel, ClassificationArgs
import pandas as pd
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)


In [ ]:
train_data = training_data[['Sentence','LABEL']]

train_data['text'] = train_data['Sentence']
train_data['labels'] = train_data['LABEL']

train_data

In [ ]:
iloc_label(df,'LABEL')
df.head()

,ID,START,LEN,Entity,Sentence,URL,LABEL
0,1,28,2,Schmitt Pál,"A magyarok tiszta, egyértelmű, vállalható, jól...",http://kecskemetitv.hu/hun/sa_kecskemet/i_belf...,1
1,2,7,2,Matolcsy György,A szóvivő megerősítette: a kabinet Matolcsy Gy...,http://mti.hu/article-proxy/2011/02/18/szijjar...,1
2,3,15,2,Szijjártó Péter,A nyugdíjasok utazási kedvezményének megvonása...,http://mti.hu/article-proxy/2011/02/18/szijjar...,1
3,4,18,2,Szabó Attila,"Itt találta el az a lövés, amellyel egy vadász...",http://kecskemetitv.hu/hun/sa_zuschlag_per_kec...,1
4,5,12,2,P. Zoltán,"Egy csempész ismerőse szintén ,,cigiben utazó""...",http://atv.hu/bunugy/20110223_felesegevel_egyu...,0


In [ ]:
test_data = testing_data[['Sentence','LABEL']]
test_data['text'] = test_data['Sentence']
test_data['labels'] = test_data['LABEL']
test_data

In [ ]:
model_args = ClassificationArgs(num_train_epochs=1)


In [ ]:
train_data

bert-base-multilingual-uncased kell felülírni a "SZTAKI-HLT/hubert-base-cc", vagy "bert-base-multilingual-cased" attól függően melyik modelt szeretnénk.

In [ ]:
model = ClassificationModel(
    "bert", "bert-base-multilingual-uncased", num_labels = 3)

Downloading:   0%|          | 0.00/625 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.64M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

In [ ]:
model.train_model(train_data)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(test_data)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/6160 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_train_bert_128_3_2


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/770 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model: Training of bert model complete. Saved to outputs/.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1540 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_3_2


Running Evaluation:   0%|          | 0/193 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.3735452983557661, 'eval_loss': 0.6056352131107311}


In [ ]:
result

{'eval_loss': 0.3967782326930545, 'mcc': 0.648168960211348}

In [ ]:
from sklearn.metrics import f1_score, accuracy_score


def f1_multiclass(labels, preds):
    return f1_score(labels, preds, average='micro')
    
result, model_outputs, wrong_predictions = model.eval_model(test_data, f1=f1_multiclass, acc=accuracy_score)

INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/1540 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_utils: Saving features into cached file cache_dir/cached_dev_bert_128_3_2


Running Evaluation:   0%|          | 0/193 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.3735452983557661, 'eval_loss': 0.6056352131107311, 'f1': 0.7746753246753245, 'acc': 0.7746753246753246}
